In [1]:
from glob import glob

import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
import tensorflow_addons as tfa

from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
import tensorflow.keras.backend as backend
import tensorflow_datasets as tfds

import boto3
import json
from sklearn.metrics import confusion_matrix

import cv2
from DBQueries import Queries as  DBQueries

import mysql.connector
import os

from flask_mysqldb import MySQL

from Config import BulkUploadConfig as Config_file

C:\Users\ashwi\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
mydb = mysql.connector.connect(
  host=Config_file.MYSQL_HOST,
  user=Config_file.MYSQL_USER,
  password=Config_file.MYSQL_PASSWORD,
  database=Config_file.MYSQL_DB
)

custom_session=boto3.session.Session(profile_name='default')
s3_client = custom_session.client('s3')


with open('Config.json') as f:
    Config = json.load(f)
Label = []
Color = []
Classes = []
Wound_Labels = []
PeriWound_Labels = []
Exudate_Labels =[]
for config_name in Config['labels']:
    Label.append(Config['labels'][config_name]['id'])
    Color.append(Config['labels'][config_name]['color'])
    if Config['labels'][config_name]['categorie'] == 'Exudate':
        Exudate_Labels.append(Config['labels'][config_name]['id'])
        Wound_Labels.append(Config['labels'][config_name]['id'])
    if Config['labels'][config_name]['categorie'] == 'Wound':
        Wound_Labels.append(Config['labels'][config_name]['id'])
    if Config['labels'][config_name]['categorie'] == 'PeriWound':
        PeriWound_Labels.append(Config['labels'][config_name]['id'])
    Classes.append(config_name)

In [24]:
for i in Wound_Labels:
    print(Classes[Label.index(i)])

Blood
Bone
DeadTissue
Eschar
HealthyGranulation
Slough
UnHealthyGranulation


In [3]:
IMG_SIZE = 128
OUTPUT_CHANNELS = len(Classes)
dataset_path = "input/"
output_path = "output/"
SEED = 42
BATCH_SIZE = 1
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[IMG_SIZE, IMG_SIZE, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = True

up_stack = []

for l in reversed(layers[:-1]):
    up_stack.append(pix2pix.upsample(l.shape[-1], 3))


def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[128, 128, 3])
    x = inputs
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])
    
    for up, skip in zip(up_stack, skips):
        x = up(x)
        attention = tf.keras.layers.AdditiveAttention()([x,skip])
        concat = tf.keras.layers.Concatenate()
        x = concat([x, attention])
    
    last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same')  #64x64 -> 128x128
    x = last(x)
    return tf.keras.Model(inputs=inputs, outputs=x)


model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #loss=TTL_Loss,
              metrics=['accuracy'])

model.load_weights('best_model_unet.h5')

In [5]:
def parse_image(img_path: str) -> dict:
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)
    return {'image': image}

@tf.function
def normalize(input_image: tf.Tensor) -> tuple:
    input_image = tf.cast(input_image, tf.float32) / 255.0
    #input_image = tf.image.rgb_to_hsv(input_image)
    return input_image

@tf.function
def load_image_unseen(datapoint: dict) -> tuple:
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    """if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)"""

    input_image = normalize(input_image)
    return input_image

def display_sample(display_list):
    plt.figure(figsize=(18, 18))
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        
        # tf.image.hsv_to_rgb(
        plt.imshow(  tf.keras.preprocessing.image.array_to_img(display_list[i]) )
        plt.axis('off')
    plt.show()

def create_mask(pred_mask: tf.Tensor) -> tf.Tensor:
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask

def show_predictions(dataset=None, num=1):
    for image in dataset.take(num):
        pred_mask = model.predict(image)
        display_sample([image[0], create_mask(pred_mask)[0]])

def save_rediction_jpg(dataset=None, num=1):
    for image in dataset.take(num):
        pred_mask = model.predict(image)
        tf.keras.preprocessing.image.array_to_img(create_mask(pred_mask)[0])

In [26]:
for img_path in os.listdir(dataset_path):
    image = cv2.imread(dataset_path+img_path)
    cv2.imwrite('output/Raw/'+img_path,image)
    print(img_path)
    unseen_dataset = tf.data.Dataset.list_files('output/Raw/'  + "*.jpg", seed=SEED)
    unseen_dataset = unseen_dataset.map(parse_image)
    dataset = {"unseen": unseen_dataset}
    dataset['unseen'] = dataset['unseen'].map(load_image_unseen)
    dataset['unseen'] = dataset['unseen'].repeat()
    dataset['unseen'] = dataset['unseen'].batch(BATCH_SIZE)
    dataset['unseen'] = dataset['unseen'].prefetch(buffer_size=AUTOTUNE)
    
    for image in dataset['unseen'] .take(1):
        wound_image = img_path.lower()
        filename = wound_image
        upload_parameters = wound_image.split("_")
        doc_id = upload_parameters[0]
        patient_id = upload_parameters[1]
        wound_id = upload_parameters[2]
        upload_date = upload_parameters[3].replace(".jpg","")+' 00:00:00'

        
        upload_id = DBQueries.create_upload_id(mydb,doc_id,patient_id,upload_date,'Bulk Upload')
        DBQueries.insert_begin_end(mydb,wound_image,upload_id,'start')
        upload_ids_to_delete = DBQueries.get_upload_with_filename(mydb,filename)
        NoUploadsDeleted = DBQueries.delete_algo_uploads(mydb,upload_ids_to_delete)
        DBQueries.insert_into_filename(mydb,upload_id,filename)
    
        s3_client.upload_file(os.path.join('output/Raw/', filename), Config_file.S3_BUCKET, 'After_Cleaning/Raw/Image/'+filename)
        
        
        pred_mask = model.predict(image)
        image = cv2.imread(dataset_path+img_path)
        image_original = image.copy()
        
        image = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
        prediction_image = create_mask(pred_mask)[0][:,:,0].numpy()
        Wound_Mask = np.full(prediction_image.shape, False)
        PeriWound_Mask =np.full(prediction_image.shape, False)
        Wound_Mask[np.isin(prediction_image, Wound_Labels)] = True
        PeriWound_Mask[np.isin(prediction_image, PeriWound_Labels)] = True
        
        
    
        image_null = np.zeros(image.shape,dtype=np.uint8)
        image_null[np.where(Wound_Mask)] = 255
        contours, hierarchy = cv2.findContours(image_null[:,:,1],cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        Wound_Image = cv2.drawContours(image, contours, -1, (146,87,97), 1)
        
        wound_predicted =[]
        TissueDistribution =[]
        
        for c in contours:
            x,y,w,h = cv2.boundingRect(c)
            wound_predicted.append({'wound_identifier':"area_"+str(x)+"_"+str(y),'wound_size_width':w,'wound_size_height':h,'wound_x_center':x,'wound_y_center':y,'wound_area':cv2.contourArea(c),'conf':1})
        
        DBQueries.insert_into_wound(mydb,wound_predicted,upload_id,wound_id,"After Cleaning")
        cv2.imwrite('output//Wound//'+img_path,Wound_Image)
        s3_client.upload_file('output//Wound//'+img_path, Config_file.S3_BUCKET, 'After_Cleaning/Algo/Wound/'+img_path)
        s3_client.upload_file(os.path.join('output/Raw/', filename), Config_file.S3_BUCKET, 'After_Cleaning/Algo/Wound_Characterstics/'+filename)
        
        image_null = np.zeros(image.shape,dtype=np.uint8)
        image_null[np.where(PeriWound_Mask)] = 255

        

        contours, hierarchy = cv2.findContours(image_null[:,:,1],cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        peri_wound_predicted = []
        PeriWound_Image = cv2.drawContours(image, contours, -1, (146,87,97), 1)
        for c in contours:
            x,y,w,h = cv2.boundingRect(c)
            peri_wound_predicted.append({'wound_identifier':"area_"+str(x)+"_"+str(y),'wound_size_width':w,'wound_size_height':h,'wound_x_center':x,'wound_y_center':y,'wound_area':cv2.contourArea(c),'conf':1})
        DBQueries.insert_into_periwound(mydb,peri_wound_predicted,upload_id,wound_id,"After Cleaning")
        
        gray = np.ones(image.shape, dtype=np.float)*(150,150,150)
        
        PeriWound_Image = gray*Wound_Mask[...,None] + PeriWound_Image*(1-.5*Wound_Mask[...,None])
        
        cv2.imwrite('output//PeriWound//'+img_path,PeriWound_Image)
        s3_client.upload_file('output//PeriWound//'+img_path, Config_file.S3_BUCKET, 'After_Cleaning/Algo/PeriWound/'+img_path)
        s3_client.upload_file('output//PeriWound//'+img_path, Config_file.S3_BUCKET, 'After_Cleaning/Algo/PeriWound_Characterstics/'+img_path)

        
         
        #Wound/PeriWound/Exudate
        FreqMatrix = np.unique(prediction_image, return_counts=True)
        Characterstics = []
        for label, freq in zip(FreqMatrix[0],FreqMatrix[1]):
            characterstic = Classes[Label.index(label)]
            Characterstics.append({'Label':characterstic, 'area':freq})
        
        print(Characterstics)
        DBQueries.insert_into_characterstics_color_model(mydb,Characterstics,upload_id)
        
        os.unlink('output/Raw/'+img_path);
        break;


doc3_p10_11_2020-09-15.JPG
[{'Label': 'HealthyGranulation', 'area': 991}, {'Label': 'UnHealthyGranulation', 'area': 32}, {'Label': 'Slough', 'area': 247}, {'Label': 'Eczema', 'area': 7}, {'Label': 'Endema', 'area': 4}, {'Label': 'HealthyEdge', 'area': 5}, {'Label': 'RolledOutEdge', 'area': 6}, {'Label': 'HealedWound', 'area': 49}, {'Label': 'Bone', 'area': 1}, {'Label': 'Maceration', 'area': 1568}, {'Label': 'Background', 'area': 11561}, {'Label': 'Apparatus', 'area': 40}, {'Label': 'Skin', 'area': 1468}, {'Label': 'Blood', 'area': 238}, {'Label': 'RolledInEdge', 'area': 30}, {'Label': 'PeriWoundDiscoloration', 'area': 30}, {'Label': 'MaceratedEdge', 'area': 107}]
doc3_p10_11_2020-09-16.JPG
[{'Label': 'HealthyGranulation', 'area': 991}, {'Label': 'UnHealthyGranulation', 'area': 32}, {'Label': 'Slough', 'area': 247}, {'Label': 'Eczema', 'area': 7}, {'Label': 'Endema', 'area': 4}, {'Label': 'HealthyEdge', 'area': 5}, {'Label': 'RolledOutEdge', 'area': 6}, {'Label': 'HealedWound', 'area':

doc3_p1_2_2020-09-05.jpg
[{'Label': 'HealthyGranulation', 'area': 531}, {'Label': 'UnHealthyGranulation', 'area': 36}, {'Label': 'Slough', 'area': 100}, {'Label': 'Callus', 'area': 120}, {'Label': 'Endema', 'area': 111}, {'Label': 'HealthyEdge', 'area': 1}, {'Label': 'RolledOutEdge', 'area': 4}, {'Label': 'HealedWound', 'area': 145}, {'Label': 'DeadTissue', 'area': 4}, {'Label': 'Maceration', 'area': 61}, {'Label': 'Background', 'area': 8515}, {'Label': 'Skin', 'area': 6251}, {'Label': 'Blood', 'area': 20}, {'Label': 'RolledInEdge', 'area': 175}, {'Label': 'PeriWoundDiscoloration', 'area': 310}]
doc3_p1_2_2020-09-06.jpg
[{'Label': 'HealthyGranulation', 'area': 636}, {'Label': 'UnHealthyGranulation', 'area': 17}, {'Label': 'Slough', 'area': 64}, {'Label': 'Callus', 'area': 243}, {'Label': 'Endema', 'area': 37}, {'Label': 'RolledOutEdge', 'area': 12}, {'Label': 'HealedWound', 'area': 189}, {'Label': 'DeadTissue', 'area': 8}, {'Label': 'Maceration', 'area': 24}, {'Label': 'Background', 'a

doc3_p2_3_2020-09-10.JPG
[{'Label': 'Eschar', 'area': 376}, {'Label': 'Endema', 'area': 760}, {'Label': 'DeadTissue', 'area': 105}, {'Label': 'Maceration', 'area': 5}, {'Label': 'Background', 'area': 9594}, {'Label': 'Apparatus', 'area': 44}, {'Label': 'Skin', 'area': 5486}, {'Label': 'RolledInEdge', 'area': 8}, {'Label': 'PeriWoundDiscoloration', 'area': 6}]
doc3_p2_3_2020-09-14.JPG
[{'Label': 'HealthyGranulation', 'area': 411}, {'Label': 'UnHealthyGranulation', 'area': 163}, {'Label': 'Slough', 'area': 164}, {'Label': 'Callus', 'area': 1}, {'Label': 'Endema', 'area': 1509}, {'Label': 'HealthyEdge', 'area': 2}, {'Label': 'RolledOutEdge', 'area': 2}, {'Label': 'HealedWound', 'area': 23}, {'Label': 'DeadTissue', 'area': 9}, {'Label': 'Bone', 'area': 4}, {'Label': 'Maceration', 'area': 4}, {'Label': 'Background', 'area': 4514}, {'Label': 'Apparatus', 'area': 13}, {'Label': 'Skin', 'area': 8951}, {'Label': 'RolledInEdge', 'area': 419}, {'Label': 'PeriWoundDiscoloration', 'area': 195}]
doc

doc3_p4_5_2020-09-11.JPG
[{'Label': 'HealthyGranulation', 'area': 935}, {'Label': 'UnHealthyGranulation', 'area': 7}, {'Label': 'Slough', 'area': 45}, {'Label': 'Callus', 'area': 28}, {'Label': 'Eczema', 'area': 5}, {'Label': 'Endema', 'area': 33}, {'Label': 'RolledOutEdge', 'area': 29}, {'Label': 'HealedWound', 'area': 233}, {'Label': 'DeadTissue', 'area': 6}, {'Label': 'Bone', 'area': 2}, {'Label': 'Maceration', 'area': 40}, {'Label': 'Background', 'area': 7144}, {'Label': 'Apparatus', 'area': 20}, {'Label': 'Skin', 'area': 7368}, {'Label': 'Blood', 'area': 37}, {'Label': 'RolledInEdge', 'area': 233}, {'Label': 'PeriWoundDiscoloration', 'area': 207}, {'Label': 'MaceratedEdge', 'area': 12}]
doc3_p4_5_2020-09-14.JPG
[{'Label': 'HealthyGranulation', 'area': 305}, {'Label': 'UnHealthyGranulation', 'area': 141}, {'Label': 'Slough', 'area': 55}, {'Label': 'Callus', 'area': 17}, {'Label': 'Eczema', 'area': 4}, {'Label': 'Endema', 'area': 39}, {'Label': 'RolledOutEdge', 'area': 33}, {'Label'

[{'Label': 'HealthyGranulation', 'area': 880}, {'Label': 'UnHealthyGranulation', 'area': 9}, {'Label': 'Slough', 'area': 252}, {'Label': 'Callus', 'area': 7}, {'Label': 'Eczema', 'area': 1}, {'Label': 'Endema', 'area': 3}, {'Label': 'RolledOutEdge', 'area': 37}, {'Label': 'HealedWound', 'area': 192}, {'Label': 'Bone', 'area': 7}, {'Label': 'Maceration', 'area': 98}, {'Label': 'Background', 'area': 11119}, {'Label': 'Apparatus', 'area': 62}, {'Label': 'Skin', 'area': 3493}, {'Label': 'Blood', 'area': 47}, {'Label': 'RolledInEdge', 'area': 116}, {'Label': 'PeriWoundDiscoloration', 'area': 35}, {'Label': 'MaceratedEdge', 'area': 26}]
doc3_p6_7_2020-09-20.jpg
[{'Label': 'Eschar', 'area': 1}, {'Label': 'HealthyGranulation', 'area': 270}, {'Label': 'UnHealthyGranulation', 'area': 9}, {'Label': 'Slough', 'area': 128}, {'Label': 'Eczema', 'area': 1}, {'Label': 'Endema', 'area': 2}, {'Label': 'RolledOutEdge', 'area': 9}, {'Label': 'HealedWound', 'area': 123}, {'Label': 'DeadTissue', 'area': 1},

doc3_p8_9_2020-09-19.JPG
[{'Label': 'Eschar', 'area': 2}, {'Label': 'HealthyGranulation', 'area': 116}, {'Label': 'UnHealthyGranulation', 'area': 16}, {'Label': 'Eczema', 'area': 3}, {'Label': 'Endema', 'area': 17}, {'Label': 'RolledOutEdge', 'area': 19}, {'Label': 'HealedWound', 'area': 14}, {'Label': 'Bone', 'area': 1}, {'Label': 'Maceration', 'area': 3}, {'Label': 'Background', 'area': 9831}, {'Label': 'Apparatus', 'area': 19}, {'Label': 'Skin', 'area': 5711}, {'Label': 'Blood', 'area': 3}, {'Label': 'RolledInEdge', 'area': 227}, {'Label': 'PeriWoundDiscoloration', 'area': 402}]
doc3_p8_9_2020-09-20.JPG
[{'Label': 'HealthyGranulation', 'area': 137}, {'Label': 'UnHealthyGranulation', 'area': 8}, {'Label': 'Slough', 'area': 24}, {'Label': 'Callus', 'area': 4}, {'Label': 'Eczema', 'area': 1}, {'Label': 'Endema', 'area': 2}, {'Label': 'RolledOutEdge', 'area': 8}, {'Label': 'HealedWound', 'area': 50}, {'Label': 'Bone', 'area': 2}, {'Label': 'Maceration', 'area': 1}, {'Label': 'Background

In [ ]:
Wound_Mask_Scaled = cv2.resize(Wound_Mask, (512,512), interpolation = inter)

In [ ]:
DBQueries.insert_into_characterstics_color_model(mydb,Characterstics,upload_id)

In [ ]:
 image.shape

In [ ]:
unseen_dataset = tf.data.Dataset.list_files(dataset_path  + "*.jpg", seed=SEED)
unseen_dataset = unseen_dataset.map(parse_image)
dataset = {"unseen": unseen_dataset}
dataset['unseen'] = dataset['unseen'].map(load_image_unseen)
dataset['unseen'] = dataset['unseen'].repeat()
dataset['unseen'] = dataset['unseen'].batch(BATCH_SIZE)
dataset['unseen'] = dataset['unseen'].prefetch(buffer_size=AUTOTUNE)